In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
filepath="./Cyberbullying_emotion/"
DATANUMBER=2000
MINLENGTH=15

##Prepare Harassement samples

In [ ]:
#Download from https://www.kaggle.com/datasets/surekharamireddy/malignant-comment-classification, only train.csv. Change name to harassement_reviews

harassment_reviews = pd.read_csv(filepath+"harassement_reviews.csv", index_col=0) #Cyberbullying datasets

In [ ]:
label=[]
tags=['malignant','highly_malignant','rude','threat','abuse','loathe'] #original tags, merge to one
for i in range(len(harassment_reviews)):
  if (harassment_reviews.iloc[i]['malignant']+harassment_reviews.iloc[i]['highly_malignant']+harassment_reviews.iloc[i]['rude']+harassment_reviews.iloc[i]['threat']+harassment_reviews.iloc[i]['abuse']+harassment_reviews.iloc[i]['loathe'])>=1:
    label.append(1)
  else:
    label.append(0)
harassment_reviews['label']=label


In [ ]:
#Tag name on the comments
!wget 'https://nlp.stanford.edu/software/stanford-ner-4.2.0.zip'
!unzip stanford-ner-4.2.0.zip
PATH_TO_JAR='/content/stanford-ner-2020-11-17/stanford-ner-4.2.0.jar'
PATH_TO_MODEL = '/content/stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz'
from nltk.tag.stanford import StanfordNERTagger

In [ ]:
#only select the data including name
collectdata_pos_index=[]
collectdata_neg_index=[]
name_pos=[]
name_neg=[]
st=StanfordNERTagger(model_filename=PATH_TO_MODEL, path_to_jar=PATH_TO_JAR, encoding='utf-8')
for i in range(len(harassment_reviews)):
  sentence=harassment_reviews['comment_text'][i]
  tokens = re.findall('\\b[A-Z][A-Za-z]+\\b',sentence)
  tags = st.tag(tokens)
  for tag in tags:
      if tag[1]=='PERSON':
        if harassment_reviews['label']==1:
            collectdata_pos_index.append(i)
            name_pos.append(tag[0])
        else:
          collectdata_neg_index.append(i)
          name_neg.append(tag[0])
        break
  if len(collectdata_pos_index)>DATANUMBER and len(collectdata_neg_index)>DATANUMBER : break

##Prepare Fake&legit samples

In [ ]:
#fake sample
fake=[]
for i in range(1,250):
  filename="{:03d}".format(i)+"fake.txt"
  file = open(filepath+"fake/"+filename, "r")
  fake.append(file.read())
  file.close()

In [ ]:
#legit sample
legit=[]
for i in range(1,250):
  filename="{:03d}".format(i)+"legit.txt"
  file = open(filepath+"legit/"+filename, "r")
  legit.append(file.read())
  file.close()

##Create HDCyberbullying dataset

In [ ]:
pos_comment=[]
for i in collectdata_pos_index:
 if len(harassment_reviews.iloc[i].comment_text)>MINLENGTH
    pos_comment.append(harassment_reviews.iloc[i].comment_text)

In [ ]:
neg_comment=[]
for i in collectdata_neg_index:
 if len(harassment_reviews.iloc[i].comment_text)>MINLENGTH
    neg_comment.append(harassment_reviews.iloc[i].comment_text)

In [ ]:
data=[]
for p in pos_comment:
  data.append([p,1])
for n in neg_comment:
  data.append([n,0])
for f in fake:
  data.append([f,2])
for l in legit:
  data.append([l,0])

In [ ]:
header = ['Comments', 'label']
HDCyberbullying = pd.DataFrame(data, columns=header)
train, test = train_test_split(HDCyberbullying, test_size=0.90)